## softmax回归的从零开始实现

In [1]:
import torch
from IPython import display
from d2l import torch as d2l

In [4]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

## 1. 初始化模型参数
由于Fashi MNIST数据有10个类别，所以网络输出维度为10。因此，权重将构成一个784 x 10的矩阵，偏置将构成一个 1x 10的行向量。

In [6]:
num_inputs = 784
num_outputs = 10

# 权重服从正态分布
w = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
# 偏置初始化为0
b = torch.zeros(num_outputs, requires_grad=True)

## 2. 定义softmax操作
softmax由三个步骤组成：
* (1) 对每个项求幂（使用exp）
* (2) 对每一行求和（小批量中每个样本是一行），得到每个样本的归一化常数
* (3) 将每一行除以其归一化常数，确保结果的和为1
$$
softmax(X)_{ij} = \frac{exp(X_{ij})}{\sum_{k}{exp(X_{ik})}}
$$

分母或归一化常数，也称为配分函数（其对数称为对数-配分函数）

In [11]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)
    return X_exp / partition;  # 应用广播机制

In [12]:
X = torch.normal(0, 1, (2, 5))
X_prob = softmax(X)
X_prob, X_prob.sum(1)

(tensor([[0.0378, 0.0730, 0.5716, 0.1752, 0.1425],
         [0.0573, 0.3576, 0.3143, 0.2316, 0.0391]]),
 tensor([1.0000, 1.0000]))

## 3. 定义模型
定义输入如何通过网络映射到输出。

In [13]:
# 将数据传递到我们的模型之前，使用reshape函数将每张原始图像展平为向量
def net(X):
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)

## 4. 定义损失函数
交叉熵损失函数： 采用真实标签的预测概率的负对数似然。
实现方式不采用Python的 for 循环迭代预测（这往往是低效的），而是通过一个运算符选择所有元素

* 创建一个数据y_hat，其中包含2个样本在3个类别的概率，他们对应的标签y
* 第一个样本中，第一类是正确的预测，而在第二个样本中，第三类是正确的预测
* 然后使用y作为y_hat中概率的索引

In [66]:
y = torch.tensor([0, 2]) 
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]

tensor([0.1000, 0.5000])

In [67]:
# 实现交叉熵损失函数
def cross_entropy(y_hat, y):
    return -torch.log(y_hat[range(len(y_hat)), y])

cross_entropy(y_hat, y)

tensor([2.3026, 0.6931])

## 5. 分类准确率
给定预测概率分布y_hat，必须输出**硬预测(hard prediction)**时，通常选择预测概率最高的类

当预测与标签分类y一致时，他们是正确的。**分类准确率即正确预测数量与总预测数量之比**

为了计算准确率，我们执行以下操作：
* 首先，如果y_hat是矩阵，那么假定第二个维度存在每个类的预测分数。使用argmax获得每行中最大元素的索引来获得预测类别

* 然后我们将预测类别与真实y元素进行比较

In [70]:
def accuracy(y_hat, y):
    """计算预测正确的数量"""
    # 第二维度存储每个类的预测分数
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        # argmax获得每行中最大元素的索引来获得预测类别
        y_hat = y_hat.argmax(axis=1)
    # 由于等式运算符 == 对数据类型很敏感，需要将比较对象的数据类型一致
    cmp = y_hat.type(y.dtype) == y 
    return float(cmp.type(y.dtype).sum())

In [71]:
accuracy(y_hat, y) / len(y)

0.5